In [1]:
import pandas as pd

# ======================================
# Path
# ======================================
csv_path = '../paper2/total/true_detections_final.csv'

# ======================================
# Read file
# ======================================
df = pd.read_csv(csv_path)

# Sort by Source ID
df = df.sort_values('Source_ID')

# Take first 10 rows
sample = df.head(10)

# Select only needed columns (order matters)
sample = sample[
    ['Source_ID', 'RA', 'DEC', 'RM', 'E_RM', 'fracPol',
     'S_reffreq', 'RMS', 'F1', 'F2']
]

# ======================================
# Formatting
# ======================================
sample['RA']        = sample['RA'].map(lambda x: f"{x:.5f}")
sample['DEC']       = sample['DEC'].map(lambda x: f"{x:.5f}")
sample['RM']        = sample['RM'].map(lambda x: f"{x:.2f}")
sample['E_RM']      = sample['E_RM'].map(lambda x: f"{x:.2f}")
sample['fracPol']   = sample['fracPol'].map(lambda x: f"{x:.2f}")
sample['S_reffreq'] = sample['S_reffreq'].map(lambda x: f"{x:.2f}")

# Convert RMS from Jy to μJy BEFORE formatting
sample['RMS'] = (sample['RMS'] * 1e6).map(lambda x: f"{x:.1f}")

# ======================================
# Print LaTeX table
# ======================================
print(r"""
\begin{table*}
\centering
\small
\setlength{\tabcolsep}{3pt}
\renewcommand{\arraystretch}{1.1}
\caption{Example rows from the machine-readable catalog of true Faraday-rotation detections,
sorted by Source ID. Units are given in the header row.}
\label{tab:true_detections_final_sample}
\begin{tabular}{lccccccccc}
\toprule
Source ID & RA & Dec & RM & dRM & fracPol & $S_{\mathrm{ref}}$ & RMS & F1 & F2 \\
          & deg & deg & rad m$^{-2}$ & rad m$^{-2}$ & \% & mJy & $\mu$Jy & flag & flag \\
\midrule
""")

for _, row in sample.iterrows():
    print(
        f"{row['Source_ID']} & "
        f"{row['RA']} & "
        f"{row['DEC']} & "
        f"{row['RM']} & "
        f"{row['E_RM']} & "
        f"{row['fracPol']} & "
        f"{row['S_reffreq']} & "
        f"{row['RMS']} & "
        f"{int(row['F1'])} & "
        f"{int(row['F2'])} \\\\"
    )

print(r"""
\bottomrule
\end{tabular}
\end{table*}
""")


The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.

\begin{table*}
\centering
\small
\setlength{\tabcolsep}{3pt}
\renewcommand{\arraystretch}{1.1}
\caption{Example rows from the machine-readable catalog of true Faraday-rotation detections,
sorted by Source ID. Units are given in the header row.}
\label{tab:true_detections_final_sample}
\begin{tabular}{lccccccccc}
\toprule
Source ID & RA & Dec & RM & dRM & fracPol & $S_{\mathrm{ref}}$ & RMS & F1 & F2 \\
          & deg & deg & rad m$^{-2}$ & rad m$^{-2}$ & \% & mJy & $\mu$Jy & flag & flag \\
\midrule

VPR J031923.9+310143.3 & 49.84941 & 31.02869 & 33.78 & 0.49 & 5.48 & 0.04 & 110.8 & 0 & 0 \\
VPR J031927.7+312115.2 & 49.86532 & 31.35421 & 58.60 & 1.46 & 7.51 & 0.01 & 36.5 & 0 & 0 \\
VPR J032003.0+310733.8 & 50.01252 & 31.12604 & 75.95 & 0.12 & 5.09 & 0.11 & 147.1 & 0 & 1 \\
VPR J032021.4+313824.5 & 50.08925 & 31.64013 & 47.79 & 0.49 & 

In [3]:
import pandas as pd
import numpy as np

# === Input/Output Files ===
input_csv        = "../paper2/total/true_detections_final.csv"
output_mrt       = "../paper2/total/true_detections_final_mrt.txt"
output_data_only = "../paper2/total/true_detections_final_data_only.txt"

# === Load Data ===
df = pd.read_csv(input_csv)
df.columns = df.columns.str.strip()

# Optional: sort by Source_ID
df = df.sort_values("Source_ID")

# === Column order for MRT (NO RA/DEC ERRORS) ===
columns = [
    "Source_ID",
    "RA",
    "DEC",
    "RM",
    "E_RM",
    "fracPol",
    "S_reffreq",
    "RMS",
    "F1",
    "F2"
]

# === Convert RMS from Jy to µJy (if RMS is currently in Jy) ===
# Comment out if RMS is already in µJy.
df["RMS"] = df["RMS"] * 1e6
df["S_reffreq"] = df["S_reffreq"] * 1e3

# === HEADER ROWS ===
labels = columns

# --- Units row ---
units = []
for col in columns:
    if col in ["RA", "DEC"]:
        units.append("deg")
    elif col in ["RM", "E_RM"]:
        units.append("rad/m2")
    elif col == "fracPol":
        units.append("%")
    elif col == "S_reffreq":
        units.append("mJy")
    elif col == "RMS":
        units.append("uJy")
    else:
        units.append("---")

# --- Explanations row ---
explanations = []
for col in columns:
    if col == "Source_ID":
        explanations.append("IAU-compliant source name")
    elif col == "RA":
        explanations.append("Right ascension (J2000) in degrees")
    elif col == "DEC":
        explanations.append("Declination (J2000) in degrees")
    elif col == "RM":
        explanations.append("Rotation measure")
    elif col == "E_RM":
        explanations.append("Uncertainty in rotation measure")
    elif col == "fracPol":
        explanations.append("Fractional polarization at reference frequency")
    elif col == "S_reffreq":
        explanations.append("Total Stokes I flux density at reference frequency")
    elif col == "RMS":
        explanations.append("Local RMS noise at reference frequency")
    elif col == "F1":
        explanations.append("Flag: source lies on diffuse Galactic emission")
    elif col == "F2":
        explanations.append("Flag: double-peaked Faraday spectrum")
    else:
        explanations.append("")

# --- AAS Format Codes ---
formats = []
for col in columns:
    if col == "Source_ID":
        formats.append("A20")
    elif col in ["RA", "DEC"]:
        formats.append("F10.5")   # 5 decimals
    elif col in ["RM", "E_RM"]:
        formats.append("F9.2")    # 2 decimals
    elif col == "fracPol":
        formats.append("F7.2")
    elif col == "S_reffreq":
        formats.append("F8.2")
    elif col == "RMS":
        formats.append("F9.2")    # <<--- 2 DECIMAL PLACES
    elif col in ["F1", "F2"]:
        formats.append("I1")
    else:
        formats.append("F8.2")

# --- Python formatting codes ---
py_formats = []
for col in columns:
    if col == "Source_ID":
        py_formats.append("s20")
    elif col in ["RA", "DEC"]:
        py_formats.append("f10.5")
    elif col in ["RM", "E_RM"]:
        py_formats.append("f9.2")
    elif col == "fracPol":
        py_formats.append("f7.2")
    elif col == "S_reffreq":
        py_formats.append("f8.2")
    elif col == "RMS":
        py_formats.append("f9.2")   # <<--- 2 decimals
    elif col in ["F1", "F2"]:
        py_formats.append("i1")
    else:
        py_formats.append("f8.2")

# === Helper for fixed-width formatting ===
def format_fixed(x, fmt):
    if pd.isna(x):
        spec = fmt[0]
        if spec in ["f", "i"]:
            width = int(fmt[1:].split(".")[0])
            return "".rjust(width)
        return ""
    spec = fmt[0]
    if spec == "s":
        width = int(fmt[1:])
        return f"{str(x):>{width}s}"
    elif spec == "i":
        width = int(fmt[1:])
        return f"{int(x):>{width}d}"
    elif spec == "f":
        width_precision = fmt[1:].split(".")
        width = int(width_precision[0])
        precision = int(width_precision[1])
        return f"{float(x):>{width}.{precision}f}"
    return str(x)

# === Format data rows ===
formatted_rows = []
for _, row in df.iterrows():
    formatted_rows.append(" ".join(format_fixed(row[col], fmt)
                                   for col, fmt in zip(columns, py_formats)))

# === Write MRT file (with header) ===
with open(output_mrt, "w") as f:
    f.write(" ".join(labels) + "\n")
    f.write(" ".join(units) + "\n")
    f.write(" ".join(explanations) + "\n")
    f.write(" ".join(formats) + "\n")
    for line in formatted_rows:
        f.write(line + "\n")

print(f"✅ Saved MRT file to: {output_mrt}")

# === Data-only table ===
with open(output_data_only, "w") as f:
    for line in formatted_rows:
        f.write(line + "\n")

print(f"✅ Saved data-only file to: {output_data_only}")

# === Format code for AAS journal submission ===
print("\n👉 Copy this into the AAS 'format code' box:\n")
print(" ".join(formats))


✅ Saved MRT file to: ../paper2/total/true_detections_final_mrt.txt
✅ Saved data-only file to: ../paper2/total/true_detections_final_data_only.txt

👉 Copy this into the AAS 'format code' box:

A20 F10.5 F10.5 F9.2 F9.2 F7.2 F8.2 F9.2 I1 I1
